In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fake_job_postings.csv to fake_job_postings.csv


In [ ]:
!pip install scikit-learn joblib flask pyngrok


In [ ]:
import pandas as pd, re, joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Load dataset
df = pd.read_csv('fake_job_postings.csv')

# Clean text function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Ensure we have the clean_description column
if 'clean_description' not in df.columns:
    print("🧹 Cleaning text column...")
    df['clean_description'] = df['description'].fillna('').apply(clean_text)

df = df.dropna(subset=['clean_description'])
df = df[df['clean_description'].str.strip() != '']

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['clean_description'])
y = df['fraudulent']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Save both model and vectorizer
joblib.dump(model, 'fake_job_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("✅ Model and Vectorizer trained and saved successfully!")


🧹 Cleaning text column...
✅ Model and Vectorizer trained and saved successfully!


In [ ]:
!ls


fake_job_model.pkl  fake_job_postings.csv  sample_data	tfidf_vectorizer.pkl


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request
import joblib

app = Flask(__name__)

# Load saved model and vectorizer
model = joblib.load('fake_job_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    job_desc = request.form['job_description']
    if not job_desc.strip():
        return render_template('index.html', error="Please enter a job description.")
    # Transform and predict
    X_input = vectorizer.transform([job_desc])
    prediction = model.predict(X_input)[0]
    prob = model.predict_proba(X_input)[0][1]

    label = "Fake Job" if prediction == 1 else "Real Job"
    confidence = round(prob * 100, 2) if prediction == 1 else round((1 - prob) * 100, 2)

    return render_template('result.html',
                           label=label,
                           confidence=confidence,
                           description=job_desc)

if __name__ == '__main__':
    app.run()


Writing app.py


In [ ]:
!ls


app.py		    fake_job_postings.csv  tfidf_vectorizer.pkl
fake_job_model.pkl  sample_data


In [ ]:
!mkdir -p templates


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Fake Job Detection</title>
<style>
        body { font-family: Arial; background: #f2f2f2; text-align: center; padding: 40px; }
        form { background: white; padding: 30px; border-radius: 10px; width: 50%; margin: auto; }
        textarea { width: 90%; height: 120px; padding: 10px; border-radius: 8px; border: 1px solid #ccc; }
        button { margin-top: 15px; padding: 10px 20px; border: none; border-radius: 8px; background-color: #007bff; color: white; cursor: pointer; }
        button:hover { background-color: #0056b3; }
        .error { color: red; }
</style>
</head>
<body>
<h2>Fake Job Detection System</h2>
<form action="/predict" method="POST">
<label for="job_description"><b>Enter Job Description:</b></label><br><br>
<textarea name="job_description" placeholder="Paste job post text here..."></textarea><br>
<button type="submit">Check Authenticity</button>
</form>
{% if error %}
<p class="error">{{ error }}</p>
{% endif %}
</body>
</html>


Writing templates/index.html


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Fake Job Detection System</title>
<style>
    body {
        font-family: 'Segoe UI', Arial, sans-serif;
        background: linear-gradient(to right, #007bff, #00c6ff);
        color: #333;
        text-align: center;
        padding: 50px;
    }
    .container {
        background: white;
        width: 60%;
        margin: auto;
        padding: 30px;
        border-radius: 15px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.2);
    }
    textarea {
        width: 90%;
        height: 120px;
        border-radius: 10px;
        border: 1px solid #ccc;
        padding: 10px;
        font-size: 15px;
    }
    button {
        margin-top: 15px;
        padding: 12px 25px;
        background-color: #28a745;
        color: white;
        border: none;
        border-radius: 10px;
        font-size: 16px;
        cursor: pointer;
    }
    button:hover {
        background-color: #218838;
    }
    .tips {
        margin-top: 30px;
        text-align: left;
        background: #f9f9f9;
        padding: 15px;
        border-radius: 10px;
    }
    footer {
        margin-top: 40px;
        font-size: 14px;
        color: #fff;
    }
    .error { color: red; }
</style>
</head>
<body>
<div class="container">
    <h2>Fake Job Detection System</h2>
    <form action="/predict" method="POST">
        <label for="job_description"><b>Enter Job Description:</b></label><br><br>
        <textarea name="job_description" placeholder="Paste job post text here..."></textarea><br>
        <button type="submit">Check Authenticity</button>
    </form>

    {% if error %}
    <p class="error">{{ error }}</p>
    {% endif %}

    <div class="tips">
        <h3>💡 Tips to Avoid Fake Jobs:</h3>
        <ul>
            <li>Be cautious of jobs that ask for payment or training fees.</li>
            <li>Check if the company has a valid website or LinkedIn page.</li>
            <li>Watch out for offers that seem “too good to be true.”</li>
            <li>Always verify the recruiter’s email domain.</li>
            <li>Don’t share personal financial information before verification.</li>
        </ul>
    </div>
</div>

<footer>
    <p>Developed by <b>Valmik Patil</b> | Project: <b>Fake Job Detection System</b></p>
</footer>
</body>
</html>


Writing templates/index.html


In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Prediction Result</title>
<style>
    body {
        font-family: 'Segoe UI', Arial, sans-serif;
        background: linear-gradient(to right, #00c6ff, #007bff);
        text-align: center;
        padding: 50px;
        color: #333;
    }
    .card {
        background: white;
        width: 60%;
        margin: auto;
        padding: 30px;
        border-radius: 15px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.2);
    }
    h2 { color: #007bff; }
    .fake { color: red; font-weight: bold; }
    .real { color: green; font-weight: bold; }
    progress {
        width: 80%;
        height: 20px;
        margin-top: 10px;
    }
    a {
        display: inline-block;
        margin-top: 20px;
        text-decoration: none;
        color: #007bff;
        font-weight: bold;
    }
    a:hover { text-decoration: underline; }
</style>
</head>
<body>
<div class="card">
    <h2>Prediction Result</h2>
    <p><b>Job Description:</b> {{ description }}</p>

    <h3>Prediction:
        {% if label == 'Fake Job' %}
        <span class="fake">{{ label }}</span>
        {% else %}
        <span class="real">{{ label }}</span>
        {% endif %}
    </h3>

    <p>Confidence: {{ confidence }}%</p>
    <progress value="{{ confidence }}" max="100"></progress>

    <a href="/">🔙 Go Back</a>
</div>
</body>
</html>


Writing templates/result.html


In [ ]:
!ls templates


index.html  result.html


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("354A7wIzV8oo6I3t1xYLDyPtsBH_4UifAjbTkMFA1pG6hLhgT")


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("354A7wIzV8oo6I3t1xYLDyPtsBH_4UifAjbTkMFA1pG6hLhgT")
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)

!python app.py

🌍 Public URL: NgrokTunnel: "https://unbalanced-galen-holdable.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Nov/2025 07:01:22] "GET / HTTP/1.1" 200 -
hello
cltr_c
Ctrl+c


Traceback (most recent call last):
  File "/content/app.py", line 33, in <module>
    app.run()
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 662, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/serving.py", line 1125, in run_simple
    srv.serve_forever()
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/serving.py", line 823, in serve_forever
    self.server_close()
  File "/usr/lib/python3.12/socketserver.py", line 714, in server_close
    super().server_close()
  File "/usr/lib/python3.12/socketserver.py", line 495, in server_close
    self.socket.close()
  File "/usr/lib/python3.12/socket.py", line 505, in close
    self._real_close()
  File "/usr/lib/python3.12/socket.py", line 499, in _real_close
    _ss.close(self)
KeyboardInterrupt
^C
